In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# sklearn imports
import sklearn.preprocessing as preprocessing
import sklearn.model_selection as model_selection
import sklearn.linear_model as linear_model
import sklearn.metrics as metrics

# imblearn imports
import imblearn.ensemble as ensemble
import imblearn.metrics as imblearn_metrics

# Read the CSV and Perform Basic Data Cleaning

In [3]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [4]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv.zip')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


# Split the Data into Training and Testing

In [5]:
# get dummies
X = pd.get_dummies(df)

# Drop the target columns
X.drop(['loan_status_high_risk', 'loan_status_low_risk'], axis=1, inplace=True)

# Create our target
y = df['loan_status']

In [6]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [7]:
# Set random state
rand = 1

In [8]:
# Split the X and y into X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, random_state=rand)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [9]:
# Create the StandardScaler instance
scaler = preprocessing.StandardScaler()

In [10]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [11]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [12]:
# Resample the training data with the BalancedRandomForestClassifier
brf = ensemble.BalancedRandomForestClassifier(n_estimators=100, random_state=rand)
brf.fit(X_train_scaled, y_train)

y_pred = brf.predict(X_test_scaled)

In [13]:
# Calculated the balanced accuracy score
print(f"Balanced Random Forest Classifier accuracy score: {metrics.balanced_accuracy_score(y_test, y_pred):.4f}")

Balanced Random Forest Classifier accuracy score: 0.7888


In [14]:
# Display the confusion matrix
print("Balanced Random Forest Classifier confusion matrix")
print(metrics.confusion_matrix(y_test, y_pred))

Balanced Random Forest Classifier confusion matrix
[[   71    30]
 [ 2146 14958]]


In [15]:
# Print the imbalanced classification report
print(imblearn_metrics.classification_report_imbalanced(y_test, y_pred, digits=4))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk     0.0320    0.7030    0.8745    0.0613    0.7841    0.6042       101
   low_risk     0.9980    0.8745    0.7030    0.9322    0.7841    0.6253     17104

avg / total     0.9923    0.8735    0.7040    0.9271    0.7841    0.6252     17205



In [16]:
# define features and importances lists
features = X_train.columns
importances = brf.feature_importances_

# get indices of ascending importance order
indices = np.argsort(importances)

# sort features and importances by reversed indices as defined above
sorted_importances = importances[indices[::-1]]
sorted_features = features[indices[::-1]]

for i in range(len(sorted_features)):
     print(f"{sorted_features[i]}: {sorted_importances[i]:.4f}")

total_rec_prncp: 0.0788
total_pymnt: 0.0588
total_pymnt_inv: 0.0563
total_rec_int: 0.0536
last_pymnt_amnt: 0.0500
int_rate: 0.0297
issue_d_Jan-2019: 0.0211
installment: 0.0198
dti: 0.0175
out_prncp_inv: 0.0169
total_bal_ex_mort: 0.0164
mths_since_recent_inq: 0.0152
issue_d_Mar-2019: 0.0151
revol_bal: 0.0149
max_bal_bc: 0.0149
out_prncp: 0.0149
mo_sin_old_rev_tl_op: 0.0149
annual_inc: 0.0148
bc_util: 0.0146
tot_hi_cred_lim: 0.0145
mo_sin_old_il_acct: 0.0139
mths_since_rcnt_il: 0.0135
total_rev_hi_lim: 0.0134
avg_cur_bal: 0.0133
bc_open_to_buy: 0.0133
il_util: 0.0131
total_bc_limit: 0.0130
all_util: 0.0130
pct_tl_nvr_dlq: 0.0130
tot_cur_bal: 0.0129
total_il_high_credit_limit: 0.0128
inq_last_12m: 0.0128
total_bal_il: 0.0120
mo_sin_rcnt_tl: 0.0110
loan_amnt: 0.0106
mths_since_recent_bc: 0.0106
num_rev_accts: 0.0103
total_acc: 0.0102
open_acc_6m: 0.0098
num_il_tl: 0.0097
num_actv_rev_tl: 0.0096
mo_sin_rcnt_rev_tl_op: 0.0094
num_bc_tl: 0.0087
num_sats: 0.0086
num_op_rev_tl: 0.0083
num_bc_sa

### Easy Ensemble Classifier

In [17]:
# Train the EasyEnsembleClassifier
eec = ensemble.EasyEnsembleClassifier(n_estimators=100, random_state=rand)
eec.fit(X_train_scaled, y_train)

y_pred = eec.predict(X_test_scaled)

In [18]:
# Calculated the balanced accuracy score
print(f"Easy Ensemble Classifier accuracy score: {metrics.balanced_accuracy_score(y_test, y_pred):.4f}")

Easy Ensemble Classifier accuracy score: 0.9316


In [19]:
# Display the confusion matrix
print("Easy Ensemble Classifier confusion matrix")
print(metrics.confusion_matrix(y_test, y_pred))

Easy Ensemble Classifier confusion matrix
[[   93     8]
 [  985 16119]]


In [20]:
# Print the imbalanced classification report
print(imblearn_metrics.classification_report_imbalanced(y_test, y_pred, digits=4))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk     0.0863    0.9208    0.9424    0.1578    0.9315    0.8659       101
   low_risk     0.9995    0.9424    0.9208    0.9701    0.9315    0.8696     17104

avg / total     0.9941    0.9423    0.9209    0.9653    0.9315    0.8696     17205

